#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.pos_tagger import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 指定语料库路径
data_file = '../data/raw/test_data.tsv'

# 读取 TSV 格式语料
# 数据存储于 DataFrame 对象
data = load_data(data_file)
print(f'成功读取 {len(data)} 条平行句对')

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print('数据前 5 行如下：')
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型英文词性标注：CLAWS7 赋码集**

In [3]:
# === 加载模型：LLM CLAWS7 赋码集 ===

# 语种：英语
lang = 'english'

# 赋码集：CLAWS7
# https://ucrel.lancs.ac.uk/claws7tags.html
tagset = 'claws'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | kimi-k2.5 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 思考模式：关闭
enable_thinking = False

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_en_claws_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
    enable_thinking=enable_thinking,
)
t1 = time.time()
print('LLM CLAWS7 英文词性标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM CLAWS7 英文词性标注模型加载完毕！
加载耗时：1.85s


In [4]:
# === 标注译文：LLM CLAWS7 赋码集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_en_claws_annos = annotate_data(
    data['target'],
    llm_en_claws_tagger,
)

# 打印标注结果
print("\n=== LLM CLAWS7 英文词性标注结果预览 ===")
display_anno(llm_en_claws_annos[0])

POS Tagging: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 425.41it/s]


=== LLM CLAWS7 英文词性标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'II'), ('a', 'AT1'), ('coastal', 'JJ'), ('road', 'NN1'), ('somewhere', 'RL'), ('south', 'RL'), ('of', 'II'), ('the', 'AT'), ('Yangtze', 'NP1'), ('River', 'NNL1'), (',', ','), ('a', 'AT1'), ('detachment', 'NN1'), ('of', 'IO'), ('soldiers', 'NN2'), (',', ','), ('each', 'PN1'), ('of', 'IO'), ('them', 'PPHO2'), ('armed', 'VVN'), ('with', 'IW'), ('a', 'AT1'), ('halberd', 'NN1'), (',', ','), ('was', 'VBDZ'), ('escorting', 'VVG'), ('a', 'AT1'), ('line', 'NN1'), ('of', 'IO'), ('seven', 'MC'), ('prison', 'NN1'), ('carts', 'NN2'), (',', ','), ('trudging', 'VVG'), ('northwards', 'RL'), ('in', 'II41'), ('the', 'II42'), ('teeth', 'II43'), ('of', 'II44'),

#### **4. 英文词性标注模型对比：LLM & CLAWS7 Tagger**

In [5]:
# === 加载 CLAWS7 标注数据 ===

# 使用网络版 CLAWS7 Tagger 标注测试数据
# https://ucrel-api.lancaster.ac.uk/claws/free.html
# 数据保存于 data/eval/claws7_tag.txt

claws_tag_file = '../data/eval/claws7_tag.txt'
c7_en_claws_annos = load_claws_tag(claws_tag_file)

# 打印标注结果
print("\n=== C7 Tagger 英文词性标注结果预览 ===")
display_anno(c7_en_claws_annos[0])


=== C7 Tagger 英文词性标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'II'), ('a', 'AT1'), ('coastal', 'JJ'), ('road', 'NN1'), ('somewhere', 'RL'), ('south', 'ND1'), ('of', 'IO'), ('the', 'AT'), ('Yangtze', 'NP1'), ('River', 'NNL1'), (',', ','), ('a', 'AT1'), ('detachment', 'NN1'), ('of', 'IO'), ('soldiers', 'NN2'), (',', ','), ('each', 'DD1'), ('of', 'IO'), ('them', 'PPHO2'), ('armed', 'VVN'), ('with', 'IW'), ('a', 'AT1'), ('halberd', 'NN1'), (',', ','), ('was', 'VBDZ'), ('escorting', 'VVG'), ('a', 'AT1'), ('line', 'NN1'), ('of', 'IO'), ('seven', 'MC'), ('prison', 'NN1'), ('carts', 'NN2'), (',', ','), ('trudging', 'VVG'), ('northwards', 'RL'), ('in', 'II'), ('the', 'AT'), ('teeth', 'NN2'), ('of', 'IO'), ('a', 

In [6]:
# === 模型对比：LLM CLAWS & C7 CLAWS ===

print('英文词性标注模型对比（CLAWS 赋码集）：LLM & C7')
compare_annos(
    llm_en_claws_annos,
    c7_en_claws_annos,
    annos_1_name="LLM",
    annos_2_name="C7",
)

英文词性标注模型对比（CLAWS 赋码集）：LLM & C7

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
Jaccard: 0.711
LLM: {('in', 'II41'), ('south', 'RL'), ('teeth', 'II43'), ('of', 'II'), ('the', 'II42'), ('each', 'PN1'), ('of', 'II44')}
C7: {('teeth', 'NN2'), ('south', 'ND1'), ('in', 'II'), ('each', 'DD1')}

[ID]: 00002
In each of the first three carts a single male prisoner was caged, identifiable by his dress as a member of the scholar class. One was a white-haired old man. The other two were men of middle years.
--------------------------------------------------------------------------------
Jaccard: 0.889
LLM: {('other', 'DA1'), ('were', 'VBR')}
C7: {('were', 'VBDR'), ('other', 'JJ')}

[ID]: 00003
The four rear carts were occupied by women, the l